### Welcome to the tennis predictor

Instructions:\
Please modify the list of players following the format, modify the match settings.\
Then run everything (little play button on the left of the cell) and go to the bottom of the script to read the results!

Repeat the above for running more simulations!

In [ ]:
# Add players and/or modify form/% of points won historically.
# Remember to change the name in all appropriate places.
Sinner = {
    "name" : "Sinner",
    "form" : 0.56
}

Alcaraz = {
    "name" : "Alcaraz",
    "form" : 0.55
}

Djokovic = {
    "name" : "Djokovic",
    "form" : 0.56
}

Tiafoe = {
    "name" : "Tiafoe",
    "form" : 0.48
}

# Who is the first player? (Make sure the player's name and form are present and accurate in the above section)
player_1 = Sinner
# Who is the second player? (Make sure the player's name and form are present and accurate in the above section)
player_2 = Alcaraz
# How many sets does one need to win in order to win the game? (Usually 2 or 3)
number_of_sets_to_win_game = 3
# How many points does one need to win in order to win a tie-break? (Usually 7 or 10)
number_of_points_to_win_tie_break = 7
# Is a 2-point advantage needed to win a tie-break? Input True if it is, False if there is no advantage.
is_there_advantage_in_the_tie_break = True


# DO NOT MODIFY ANYTHING BELOW THIS LINE! PLEASE GO BACK TO THE INSTRUCTIONS AT THE TOP.
###########################################################################################################################################
###########################################################################################################################################
###########################################################################################################################################

import numpy as np
import math

choose = math.comb

def deuce(p):
    '''
    Calculates the probability of a player winning a deuce.
    Args:
        p (float): probability of a player winning a point.
    Returns:
        float: probability of the player winning the deuce.
    '''
    # Using manually computed terminal distribution
    return p**2 / (p**2 + (1  - p)**2)

def clash(p, pts_to_reach, adv):
    '''
    Calculates the probability of a player winning a clash, which is race to "pts_to_reach" points with or without a 2-point advantage.
    Args:
        p (float): probability of the player winning a point.
        pts_to_reach (int): points to reach to win the clash.
        adv (bool): If True, a 2-point advantage is needed to win.
        If False,  no advantage is needed to win.
    Returns:
        float: probability of the player winning the clash.
    '''
    # Retrieve the probability of a player winning a deuce.
    p_deuce = deuce(p)
    prob_wins = 0
    for i in range(pts_to_reach):
        prob_wins += p ** pts_to_reach * (1 - p) ** i * choose(pts_to_reach + i - 1, i)
    if adv == True:
        prob_wins = prob_wins - p**pts_to_reach * (1 - p)**(pts_to_reach - 1) * choose(2 * pts_to_reach - 2, pts_to_reach - 1) + p**(pts_to_reach - 1) * (1 - p)**(pts_to_reach - 1) * choose(2 * pts_to_reach - 2, pts_to_reach - 1) * p_deuce
    return prob_wins

def game(p):
    '''
    Calculates the probability of a player winning a game.
    Args:
        p (float): probability of the player winning a point.
    Returns:
        float: probability of the player winning the game.
    '''
    # We use the fact that a game is a specific type of clash.
    return clash(p, 4, True)

def tie_break(p, pts_to_reach, adv):
    '''
    Calculates the probability of a player winning a tie-break.
    Args:
        p (float): probability of the player winning a point.
        pts_to_reach (int): points to reach to win the tie-break.
        adv (bool): If True, a 2-point advantage is needed to win.
        If False,  no advantage is needed to win.
    Returns:
        float: probability of the player winning the clash.
    '''
    # We use the fact that a tie-break is a specific type of clash.
    return clash(p, pts_to_reach, adv)

def set(prob_wins_pt, tb_pts, adv_tb):
    '''
    Calculates the probability of a player winning a set.
    Args:
        prob_wins_pt (float): probability of the player winning a point.
        tb_pts (int): points to reach to win the tie-break.
        adv_tb (bool): If True, a 2-point advantage is needed to win a tie-break.
        If False, no advantage is needed to win the tie-break.
    Returns:
        float: probability of the player winning the clash.
    '''
    # We retrieve the probabilities of a player winning a game and a tie-break.
    p = game(prob_wins_pt)
    p_tie = tie_break(prob_wins_pt, tb_pts, adv_tb)

    # We construct a transition matrix
    trans = np.zeros((41, 41), dtype=np.float64)
    for i in range(30):
        trans[i, i + 6] = p
    for i in range(35):
        if (i+1) % 6 == 0:
            trans[i, 40] = 1 - p
        else:
            trans[i, i + 1] = 1 - p
    for i in range(5):
        trans[30 + i, 39] = p
    trans[35, 36], trans[37, 38], trans[36, 39]  = p, p, p
    trans[35, 37], trans[36, 38], trans[37, 40] = 1 - p, 1 - p, 1 - p
    trans[39, 39], trans[40, 40] = 1, 1
    trans[38, 39], trans[38, 40] = p_tie, 1 - p_tie

    # We return the terminal probability of the player winning the set.
    return np.linalg.matrix_power(trans, 10000)[0, 39]

def match(player_1, player_2, num_sets_to_win=3, tb_pts=7, adv_tb=True):
    '''
    Calculates the players' probabilities of winning the match.
    Args:
        player_1, player_2 (dict): contains the player's "name" and "form".
        num_sets_to_win (int, optional): number of sets to reach to win. Defaults to 3.
        tb_points (int, optional): points to reach to win the tie-break. Defaults to 7.
        adv_tb (bool, optional): If True, a 2-point advantage is needed to win a tie-break.
        If False, no advantage is needed to win the tie-break. Defaults to True.
    Prints:
        str: summary of probabilities of winning for both players.
    '''
    # We extract the probability of player 1 winning a point from the relative players' form.
    prob_1_wins_pt = player_1["form"] / (player_1["form"] + player_2["form"])

    # We find out the probability of player 1 winning a set.
    prob_1_wins_set = set(prob_1_wins_pt, tb_pts, adv_tb)

    # We find and round the chance that player 1 wins the match by using the fact that a match is a specific type of clash.
    prob_1 = round(100 * clash(prob_1_wins_set, num_sets_to_win, False), 2)
    prob_2 = round(100 - prob_1, 2)
    
    # We print a summary of the probabilities of each player winning the game.
    print(f"{player_1['name']} has a {prob_1}% chance while {player_2['name']} has a {prob_2}% chance of winning.")

# We run the match probabilites and print the output.
match(player_1, player_2, number_of_sets_to_win_game, number_of_points_to_win_tie_break, is_there_advantage_in_the_tie_break)

Sinner has a 78.38% chance while Alcaraz has a 21.62% chance of winning.
